## Objetivo

O objetivo desse tutorial é introduzí-los a coloeta de dados na web utilizando a ferramenta Scrapy.

Vamos mostrar um exemplo simples de coleta utilizando o scrapy já pronto, e etão vocês vão ter de construir suas próprias "aranhas" (spiders) e parses para realizarem a coleta em um site com dados do IBGE que criamos especialmente para esssa aula.


# Scrapy

É um **framework** de código aberto e colaborativo para extração de dados a partir de websites. Ele escala bem, é simples de utilizar e facilmente extensível.



In [ ]:
#